# A little extra!

## New addition to Week 1

### The Unreasonable Effectiveness of the Agent Loop

# What is an Agent?

## Three competing definitions

1. AI systems that can do work for you independently - Sam Altman

2. A system in which an LLM controls the workflow - Anthropic

3. An LLM agent runs tools in a loop to achieve a goal

## The third one is the new, emerging definition

But what does it mean?

Let's make it real.

In [7]:
# Start with some imports - rich is a library for making formatted text output in the terminal

from rich.console import Console
from dotenv import load_dotenv
from openai import OpenAI
import json
import os
load_dotenv(override=True)

True

In [8]:
def show(text):
    try:
        Console().print(text)
    except Exception:
        print(text)

In [9]:
DEEPSEEK_URL="https://api.deepseek.com/v1"
openai = OpenAI(base_url=DEEPSEEK_URL,api_key=os.getenv("DEEPSEEK_API_KEY"))

In [10]:
# Some lists!

todos = []
completed = []

In [11]:
def get_todo_report() -> str:
    result = ""
    for index, todo in enumerate(todos):
        if completed[index]:
            result += f"Todo #{index + 1}: [green][strike]{todo}[/strike][/green]\n"
        else:
            result += f"Todo #{index + 1}: {todo}\n"
    show(result)
    return result

In [24]:
get_todo_report()

Todo #1: Define the problem: Two trains traveling toward each other from Boston and New York
Todo #2: Determine the distance between Boston and New York
Todo #3: Set up variables and equations for the problem
Todo #4: Solve for the meeting time
Todo #5: Convert the solution to a readable time format

'Todo #1: [green][strike]Define the problem: Two trains traveling toward each other from Boston and New York[/strike][/green]\nTodo #2: [green][strike]Determine the distance between Boston and New York[/strike][/green]\nTodo #3: [green][strike]Set up variables and equations for the problem[/strike][/green]\nTodo #4: [green][strike]Solve for the meeting time[/strike][/green]\nTodo #5: [green][strike]Convert the solution to a readable time format[/strike][/green]\n'

In [13]:
def create_todos(descriptions: list[str]) -> str:
    todos.extend(descriptions)
    completed.extend([False] * len(descriptions))
    return get_todo_report()

In [14]:
def mark_complete(index: int, completion_notes: str) -> str:
    if 1 <= index <= len(todos):
        completed[index - 1] = True
    else:
        return "No todo at this index."
    Console().print(completion_notes)
    return get_todo_report()

In [15]:
todos, completed = [], []

create_todos(["Buy groceries", "Finish extra lab", "Eat banana"])

Todo #1: Buy groceries
Todo #2: Finish extra lab
Todo #3: Eat banana

'Todo #1: Buy groceries\nTodo #2: Finish extra lab\nTodo #3: Eat banana\n'

In [16]:
mark_complete(1, "bought")

bought

Todo #1: Buy groceries
Todo #2: Finish extra lab
Todo #3: Eat banana

'Todo #1: [green][strike]Buy groceries[/strike][/green]\nTodo #2: Finish extra lab\nTodo #3: Eat banana\n'

In [17]:
create_todos_json = {
    "name": "create_todos",
    "description": "Add new todos from a list of descriptions and return the full list",
    "parameters": {
        "type": "object",
        "properties": {
            "descriptions": {
                'type': 'array',
                'items': {'type': 'string'},
                'title': 'Descriptions'
                }
            },
        "required": ["descriptions"],
        "additionalProperties": False
    }
}

In [18]:
mark_complete_json = {
    "name": "mark_complete",
    "description": "Mark complete the todo at the given position (starting from 1) and return the full list",
    "parameters": {
        'properties': {
            'index': {
                'description': 'The 1-based index of the todo to mark as complete',
                'title': 'Index',
                'type': 'integer'
                },
            'completion_notes': {
                'description': 'Notes about how you completed the todo in rich console markup',
                'title': 'Completion Notes',
                'type': 'string'
                }
            },
        'required': ['index', 'completion_notes'],
        'type': 'object',
        'additionalProperties': False
    }
}

In [19]:
tools = [{"type": "function", "function": create_todos_json},
        {"type": "function", "function": mark_complete_json}]

In [20]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [21]:
def loop(messages):
    done = False
    while not done:
        response = openai.chat.completions.create(model="deepseek-chat", messages=messages, tools=tools, reasoning_effort="none")
        finish_reason = response.choices[0].finish_reason
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    show(response.choices[0].message.content)

In [22]:
system_message = """
You are given a problem to solve, by using your todo tools to plan a list of steps, then carrying out each step in turn.
Now use the todo list tools, create a plan, carry out the steps, and reply with the solution.
If any quantity isn't provided in the question, then include a step to come up with a reasonable estimate.
Provide your solution in Rich console markup without code blocks.
Do not ask the user questions or clarification; respond only with the answer after using your tools.
"""
user_message = """"
A train leaves Boston at 2:00 pm traveling 60 mph.
Another train leaves New York at 3:00 pm traveling 80 mph toward Boston.
When do they meet?
"""
messages = [{"role": "system", "content": system_message}, {"role": "user", "content": user_message}]

In [23]:
todos, completed = [], []
loop(messages)

Todo #1: Define the problem: Two trains traveling toward each other from Boston and New York
Todo #2: Determine the distance between Boston and New York
Todo #3: Set up variables and equations for the problem
Todo #4: Solve for the meeting time
Todo #5: Convert the solution to a readable time format

**Problem Definition:**
- Train A leaves Boston at 2:00 pm traveling at 60 mph toward New York
- Train B leaves New York at 3:00 pm traveling at 80 mph toward Boston
- They are traveling toward each other
- Need to find when they meet

Todo #1: Define the problem: Two trains traveling toward each other from Boston and New York
Todo #2: Determine the distance between Boston and New York
Todo #3: Set up variables and equations for the problem
Todo #4: Solve for the meeting time
Todo #5: Convert the solution to a readable time format

**Distance between Boston and New York:**
The actual distance between Boston and New York is approximately **215 miles** (this is a standard estimate for 
such problems).

I'll use this reasonable estimate since no distance was provided in the problem.

Todo #1: Define the problem: Two trains traveling toward each other from Boston and New York
Todo #2: Determine the distance between Boston and New York
Todo #3: Set up variables and equations for the problem
Todo #4: Solve for the meeting time
Todo #5: Convert the solution to a readable time format

...

**Solving the equation:**
60t + 80(t-1) = 215
60t + 80t - 80 = 215
140t - 80 = 215
140t = 295
t = 295/140
t = 2.10714 hours

This means they meet **2.10714 hours** after 2:00 pm.

Todo #1: Define the problem: Two trains traveling toward each other from Boston and New York
Todo #2: Determine the distance between Boston and New York
Todo #3: Set up variables and equations for the problem
Todo #4: Solve for the meeting time
Todo #5: Convert the solution to a readable time format

**Converting to readable time:**
2.10714 hours = 2 hours + (0.10714 × 60) minutes
0.10714 × 60 = 6.4284 minutes ≈ 6 minutes and 26 seconds

Starting from 2:00 pm:
2:00 pm + 2 hours = 4:00 pm
4:00 pm + 6 minutes = 4:06 pm
4:06 pm + 26 seconds ≈ **4:06:26 pm**

**Final Answer:** The trains meet at approximately **4:06 pm**.

Todo #1: Define the problem: Two trains traveling toward each other from Boston and New York
Todo #2: Determine the distance between Boston and New York
Todo #3: Set up variables and equations for the problem
Todo #4: Solve for the meeting time
Todo #5: Convert the solution to a readable time format

## Solution

**The two trains meet at approximately 4:06 pm.**

**Step-by-step explanation:**

1. **Distance:** Using the standard distance between Boston and New York of **215 miles** (since no distance was 
provided in the problem).

2. **Time variables:** Let t = hours after 2:00 pm when they meet.

3. **Equation setup:**
   - Train from Boston: travels t hours at 60 mph → distance = 60t
   - Train from New York: leaves at 3:00 pm (1 hour later), so travels (t-1) hours at 80 mph → distance = 80(t-1)
   - When they meet: 60t + 80(t-1) = 215

4. **Solving:**
   60t + 80t - 80 = 215
   140t = 295
   t = 295/140 = 2.10714 hours

5. **Time conversion:**
   2.10714 hours = 2 hours + 0.10714 × 60 minutes
   = 2 hours + 6.4284 minutes
   ≈ 2 hours, 6 minutes, 26 seconds

6. **Meeting time:** 2:00 pm + 2 hours, 6 minutes, 26 seconds = **4:06:26 pm** ≈ **4:06 pm**

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Now try to build an Agent Loop from scratch yourself!<br/>
            Create a new .ipynb and make one from first principles, referring back to this as needed.<br/>
            It's one of the few times that I recommend typing from scratch - it's a very satisfying result.
            </span>
        </td>
    </tr>
</table>